## Machine Unlearning (MU) for Forget me not algorithm. 

 #### **1: Environment Setup**
 
In this section, we set up our Python environment and install the necessary packages. For reproducibility, it’s best to use a virtual environment.


**Prerequisities**

Ensure conda is installed on your system. You can install Miniconda or Anaconda:

* Miniconda (recommended): https://docs.conda.io/en/latest/miniconda.html

* Anaconda: https://www.anaconda.com/products/distribution

After installing conda, ensure it is available in your PATH by running. You may require to restart the terminal session:



Before installing the unlearn_diff package, follow these steps to set up your environment correctly. These instructions ensure compatibility with the required dependencies, including Python, PyTorch, and ONNX Runtime.


**Step-by-Step Setup:**

1. Create a Conda Environment Create a new Conda environment named myenv with Python 3.8.5:

```bash
conda create -n myenv python=3.8.5
```

2. Activate the Environment Activate the environment to work within it:

```bash
conda activate myenv
```

3. Install Core Dependencies Install PyTorch, torchvision, CUDA Toolkit, and ONNX Runtime with specific versions:

```bash
conda install pytorch==1.11.0 torchvision==0.12.0 cudatoolkit=11.3 onnxruntime==1.16.3 -c pytorch -c conda-forge
```

4. Install our unlearn_diff Package using pip:

```bash
pip install unlearn_diff
```

5. Install Additional Git Dependencies:

 After installing unlearn_diff, install the following Git-based dependencies in the same Conda environment to ensure full functionality:
```bash
pip install git+https://github.com/CompVis/taming-transformers.git@master git+https://github.com/openai/CLIP.git@main git+https://github.com/crowsonkb/k-diffusion.git git+https://github.com/cocodataset/panopticapi.git git+https://github.com/Phoveran/fastargs.git@main git+https://github.com/boomb0om/text2image-benchmark
```


#### **2. Downloading the Dataset**

After you install the package, you can use the following commands to download.


1. quick_canvas:

* Sample: 

```bash 
     download_data sample quick_canvas
```

* Full: 

```bash 
     download_data full quick_canvas
```

In [ ]:
#downloading sample quick_canvas dataset:

!download_data sample quick_canvas

Once you have downloaded datasets, verify the Downloaded Files.

* ls data/i2p-dataset/sample/

* ls -lh ./data/quick-canvas-dataset/sample/

#### **3. Downloading models**

* compvis: 

```bash 
     download_model compvis
```

* diffuser: 

```bash
     download_model diffuser
```


* Download best.onnx model

     ```bash
     download_best_onnx
     ```

In [ ]:
# download compvis model

!download_model compvis 

In [ ]:
# download diffuser model

!download_model diffuser

In [ ]:
!download_best_onnx

#### 4. **Train a Text Inversion**

The default configuration for training is provided by forget_me_not_train_ti_mu. You can run the training with the default settings as follows:

```python
from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu
)
algorithm.run(train_type="train_ti")
```

<br> <br>

**Overriding the Default Configuration**

If you need to override the existing configuration settings, you can specify your custom parameters (such as ckpt_path and raw_dataset_dir) directly when initializing the algorithm. For example:

```python
from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50",
    raw_dataset_dir=(
        "/home/ubuntu/Projects/balaram/packaging/data/quick-canvas-dataset/sample"
    ), 
    steps=10,
    use_sample = True
)
algorithm.run(train_type="train_ti")
```
<br><br><br>


Before performing machine unlearning, make sure to generate the safetensors weights file. This file should be produced as part of the unlearning process so that the subsequent machine unlearning step can use these weights. Once generated, the safetensors file will be
referenced via the 'ti_weights_path' parameter.

In [ ]:
#generate safetensors

from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50", #replace it with your ckpt path
    raw_dataset_dir=(
        "data/quick-canvas-dataset/sample"
    ), 
    steps=10,
    template_name = "Abstractionism",
    dataset_type = "unlearncanvas"
)
algorithm.run(train_type="train_ti")

In [ ]:
# machine unlearning using the generated safetensors using unlearn canvas dataset

from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_attn_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_attn_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50", #replace it with your ckpt path
    raw_dataset_dir=(
        "data/quick-canvas-dataset/sample"
    ),
    steps=10,
    ti_weights_path="outputs/forget_me_not/ti_models/step_inv_10.safetensors",
    devices = "0"
)
algorithm.run(train_type="train_attn")

#### Evaluation of MU

The evaluation framework is used to assess the performance of models after applying machine unlearning.

config descriptions:

* erase_diff_evaluation_config : default evaluation config for erase_dif
* ckpt_path: finetuned model path for erase_diff algorithm. 
* classifier_ckpt_path: Path to classifier model. Download the classifier ckpt from here: 
    https://drive.google.com/drive/folders/1AoazlvDgWgc3bAyHDpqlafqltmn4vm61 

* refrence_dir: original dataset dir.

In [ ]:
from mu.algorithms.forget_me_not import ForgetMeNotEvaluator
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_evaluation_config
)
from evaluation.metrics.accuracy import accuracy_score
from evaluation.metrics.fid import fid_score



evaluator = ForgetMeNotEvaluator(
    forget_me_not_evaluation_config,
    ckpt_path="outputs/forget_me_not/finetuned_models/Abstractionism",
)
generated_images_path = evaluator.generate_images()

reference_image_dir = "data/quick-canvas-dataset/sample"

accuracy = accuracy_score(gen_image_dir=generated_images_path,
                        dataset_type = "unlearncanvas",
                        classifier_ckpt_path = "models/classifier_ckpt_path/style50_cls.pth", #replace it with your ckpt path
                        reference_dir=reference_image_dir,
                        forget_theme="Bricks",
                        seed_list = ["188"] )
print(accuracy['acc'])
print(accuracy['loss'])

fid, _ = fid_score(generated_image_dir=generated_images_path,
                reference_image_dir=reference_image_dir )

print(fid)

#### **Run Attacks**

Before running attacks, download dataset for attack. Run the following command in terminal.

<span style="color:grey;"> **generate_attack_dataset --prompts_path data/prompts/nudity_sample.csv --concept i2p_nude --save_path outputs/dataset --num_samples 1** </span>

Note: If you want to generate image using full prompt then use `data/prompts/nudity.csv` as prompts_path.

Here, prompts_path is the path of csv containing prompt, concept is the name of the concept for organizing output file, save_path is the directory where generated images and metadata will be saved, num_samples is the number of images to generate per prompt.

1. **Hard Prompt Attack - Diffuser**

Use the following code if you wish to run the hard prompt attack using diffuser model.

In [ ]:
from mu_attack.configs.nudity import hard_prompt_esd_nudity_P4D_diffusers_config
from mu_attack.execs.attack import MUAttack

def run_attack_for_nudity():

    overridable_params = {
       "task.diffusers_model_name_or_path" : "outputs/forget_me_not/finetuned_models/Abstractionism", #path to the finetuned model
        "task.dataset_path" : "outputs/dataset/i2p_nude", #generated images path using above given command
        "logger.json.root" :"results/hard_prompt_esd_nudity_P4D_abstractionism_diffuser",
        "attacker.iteration": 1
    }

    MUAttack(
        config=hard_prompt_esd_nudity_P4D_diffusers_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()

**Random Attack - Diffuser**

Use the following code if you wish to run the random attack using the Diffuser model:

In [ ]:
from mu_attack.configs.nudity import no_attack_esd_nudity_classifier_diffusers_config
from mu_attack.execs.attack import MUAttack

def run_no_attack_for_nudity():

    overridable_params = {
    "task.diffusers_model_name_or_path" :"outputs/forget_me_not/finetuned_models/Abstractionism",
    "task.dataset_path" : "outputs/dataset/i2p_nude",
    "logger.json.root" : "results/no_attack_esd_nudity_P4D_abstrctionism_diffuser",
    "attacker.no_attack.dataset_path" : "outputs/dataset/i2p_nude"
    }

    MUAttack(
        config=no_attack_esd_nudity_classifier_diffusers_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_no_attack_for_nudity()

#### Evaluation of MU_attack

Calculate the performance of mu_attack using logs and images generated during attack.


In [ ]:
#import evaluation metrics from evalaution

from evaluation.metrics.asr import asr_score
from evaluation.metrics.clip import clip_score
from evaluation.metrics.fid import fid_score

root = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d"
root_no_attack = "results/no_attack_esd_nudity_P4D_abstrctionism/NoAttackEsdNudity"
devices = "0"
image_path = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d/images"
log_path = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d/log.json"
ref_batch_path = "results/no_attack_esd_nudity_P4D_abstrctionism/NoAttackEsdNudity/images"
sample_batch_path = "/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/data/i2p/nude"

asr_val = asr_score(root, root_no_attack)
print(asr_val)

clip_val = clip_score(image_path, log_path, devices)
print(clip_val)

fid_val, _ = fid_score(sample_batch_path,ref_batch_path)
print(fid_val)


### Mu Defense (AdvUnlean)

After performing unlearning and attack, we need to perform adversarial unlearning by integrating a soft prompt attack into the training loop. use the following code snippet for advunlearn.



In [ ]:
from mu_defense.algorithms.adv_unlearn.algorithm import AdvUnlearnAlgorithm
from mu_defense.algorithms.adv_unlearn.configs import adv_unlearn_config


def mu_defense():

    mu_defense = AdvUnlearnAlgorithm(
        config=adv_unlearn_config,
        diffusers_model_name_or_path = "outputs/forget_me_not/finetuned_models/Abstractionism", #finetuned model
        attack_step = 2,
        backend = "diffusers",
        attack_method = "fast_at",
        # train_method = "text_encoder_full",  #training method. check for docs for available train_method 
        train_method = "noxattn", #training method. check for docs for available train_method 
        warmup_iter = 1,
        iterations = 1
    )
    mu_defense.run()

if __name__ == "__main__":
    mu_defense()


### Evaluation for mu_defense


Description of params used:

* config: default train config for image generation.
* target_ckpt: Model ckpt after running mu_defense (AdvUnleran).
* save_path: output dir to save generated images.
* prompts_path: path to the csv with prompts.
* num_samples: number of samples to be generated for a prompt.
* folder_suffix: suffix for folder name for save path.
* devices: devices to be used.  



Note: Before performing evalaution:
1. Download coco 10k dataset from this link : https://drive.google.com/file/d/1Qgm3nNhp6ykamszN_ZvofvuzjryTsPHB/view 


In [ ]:
from mu_defense.algorithms.adv_unlearn import MUDefenseEvaluator
from mu_defense.algorithms.adv_unlearn.configs import mu_defense_evaluation_config
from mu.algorithms.erase_diff.configs import erase_diff_train_mu
from evaluation.metrics.clip import clip_score
from evaluation.metrics.fid import fid_score

target_ckpt = "outputs/results_with_retaining/nudity/coco_object/pgd/AttackLr_0.001/text_encoder_full/all/prefix_k/AdvUnlearn-nudity-method_text_encoder_full_all-Attack_pgd-Retain_coco_object_iter_1.0-lr_1e-05-AttackLr_0.001-prefix_k_adv_num_1-word_embd-attack_init_latest-attack_step_30-adv_update_1-warmup_iter_200/models/TextEncoder-text_encoder_full-epoch_1.pt"
evaluator = MUDefenseEvaluator(config=mu_defense_evaluation_config,
                            target_ckpt = target_ckpt,
                            model_config_path = erase_diff_train_mu.model_config_path,
                            save_path = "outputs/adv_unlearn/results",
                            prompts_path = "data/prompts/sample_prompt.csv",
                            num_samples = 1,
                            folder_suffix = "imagenette",
                            devices = "0",)

gen_image_path = evaluator.generate_images() #Genereate sample images before evalaution 
print(gen_image_path)  

prompt_path = "data/prompts/sample_prompt.csv"
ref_image_path = "/home/ubuntu/Projects/Palistha/msu_unlearningalgorithm/coco_dataset/extracted_files/coco_sample"
device = "0"
clip_val = clip_score(gen_image_path, prompt_path, device)    
print(clip_val)    

fid_val, _  = fid_score(gen_image_path, ref_image_path)
print(fid_val)